In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow

In [2]:
import tensorflow as tf
print(tf.__version__)

2.9.0


## Dataset formatting

In [4]:
# Define the path to the directory containing the images
image_dir = '../chest_xray/train'

# Define the batch size and image size for the dataset
batch_size = 32
img_size = (224, 224)

# Use the tf.keras.preprocessing.image_dataset_from_directory function to create the dataset
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    image_dir,
    batch_size=batch_size,
    image_size=img_size,
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset='training'
)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    image_dir,
    batch_size=batch_size,
    image_size=img_size,
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset='validation'
)

Found 5232 files belonging to 2 classes.
Using 4186 files for training.
Metal device set to: Apple M1 Pro


2023-04-06 11:10:55.541907: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-04-06 11:10:55.542559: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Found 5232 files belonging to 2 classes.
Using 1046 files for validation.
